In [7]:
import polars as pl

In [8]:
df = pl.read_csv('../outputs/csvs/lb_avax_usdc_1695057880.csv')
df.head(5)

,bin_id,reserveX,reserveY,bin_price,reserveX_in_Y
i64,i64,f64,f64,f64,f64
63,8375753,0.0,747.200137,7.005509,0.0
64,8375754,0.0,734.506927,7.01952,0.0
65,8375755,0.0,731.568879,7.033559,0.0
66,8375756,0.0,728.630832,7.047627,0.0
67,8375757,0.0,725.937622,7.061722,0.0


In [20]:
active_price = list(df.filter((pl.col("reserveX") > 0) & (pl.col("reserveY") > 0))["bin_price"])[0]
active_price